# RAG 多檔案與元數據

本範例展示：
1. **第1個儲存格**：建立多檔案向量資料庫（含元數據）
2. **第2個儲存格**：使用元數據過濾查詢

學習目標：理解如何處理多個來源的文件並使用元數據進行過濾檢索

In [ ]:
# 第1個儲存格：建立多檔案向量資料庫（含元數據）

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
books_dir = os.path.join(current_dir, "books")
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

print(f"書籍目錄: {books_dir}")
print(f"持久化目錄: {persistent_directory}")

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量存儲...")

    # 確保書籍目錄存在
    if not os.path.exists(books_dir):
        raise FileNotFoundError(
            f"目錄 {books_dir} 不存在。請檢查路徑。"
        )

    # 列出目錄中所有文字檔案
    book_files = [f for f in os.listdir(books_dir) if f.endswith(".txt")]

    # 從每個檔案讀取文字內容並儲存元數據
    documents = []
    for book_file in book_files:
        file_path = os.path.join(books_dir, book_file)
        loader = TextLoader(file_path)
        book_docs = loader.load()
        for doc in book_docs:
            # 為每個文件添加元數據以指示其來源
            doc.metadata = {"source": book_file}
            documents.append(doc)

    # 將文件分割成塊
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print("\n--- 文件塊資訊 ---")
    print(f"文件塊數量: {len(docs)}")

    # 建立嵌入模型
    print("\n--- 正在建立嵌入 ---")
    embeddings = HuggingFaceEmbeddings(
        model_name="jinaai/jina-embeddings-v2-base-zh"
    )
    print("\n--- 完成建立嵌入 ---")

    # 建立並持久化向量存儲
    print("\n--- 正在建立並持久化向量存儲 ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- 完成建立並持久化向量存儲 ---")

else:
    print("向量存儲已存在。無需初始化。")

In [ ]:
# 第2個儲存格：使用元數據過濾查詢

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 定義使用者的問題
query = "路由器的WiFi密碼如何更改？"

# 根據查詢檢索相關文件
# search_type="similarity_score_threshold": 使用相似度分數閾值過濾
# k=3: 返回最多 3 個相關文件
# score_threshold=0.1: 較低的閾值，可以返回更多相關文件
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.1},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    print(f"來源: {doc.metadata['source']}\n")

# 進階查詢：只從特定來源檢索
print("\n--- 只從《路由器設定手冊》檢索 ---")
# 使用 filter 參數指定元數據條件
specific_docs = db.similarity_search(
    query,
    k=3,
    filter={"source": "路由器設定手冊.txt"}
)

for i, doc in enumerate(specific_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    print(f"來源: {doc.metadata['source']}\n")

In [ ]:
# 第3個儲存格：整合 Chain - 多來源文件的智慧問答鏈

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# 定義使用者的問題
question = "如何保養洗衣機和路由器？"

print(f"❓ 使用者問題: {question}\n")

# 步驟 1: 從多個來源檢索相關文件
print("=" * 70)
print("步驟 1: 從多個來源檢索相關文件")
print("=" * 70)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 取4個相關文件（可能來自不同手冊）
)

retrieved_docs = retriever.invoke(question)

print(f"✅ 找到 {len(retrieved_docs)} 個相關文件\n")
for i, doc in enumerate(retrieved_docs, 1):
    source = doc.metadata.get('source', 'Unknown')
    print(f"📄 文件 {i} - 來源: {source}")
    print(f"   內容: {doc.page_content[:80]}...\n")

# 步驟 2: 建立帶有來源標註的 RAG Chain
print("=" * 70)
print("步驟 2: 建立多來源 RAG Chain")
print("=" * 70)

# 定義提示模板（強調來源資訊）
template = """你是一個專業的產品客服助手。請根據以下多個產品手冊的參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

回答要求：
1. 針對不同產品分別說明（如果問題涉及多個產品）
2. 在回答中標註資訊來源（例如：「根據洗衣機使用說明...」）
3. 只根據參考資料回答，不要編造內容
4. 如果某個產品的資訊不足，請說明
5. 用條列式整理，讓使用者容易閱讀

回答："""

prompt = ChatPromptTemplate.from_template(template)

# 建立 LLM
llm = ChatOllama(model="llama3.2", temperature=0)

# 定義文件格式化函數（包含來源資訊）
def format_docs_with_source(docs):
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"【來源 {i}: {source}】\n{doc.page_content}")
    return "\n\n---\n\n".join(formatted)

# 建立完整的 RAG Chain（整合 3_chains 的技巧）
rag_chain = (
    {
        "context": retriever | format_docs_with_source, 
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ 多來源 RAG Chain 建立完成\n")

# 步驟 3: 執行 Chain 並取得整合答案
print("=" * 70)
print("步驟 3: 執行 Chain 取得整合答案")
print("=" * 70)

answer = rag_chain.invoke(question)

print(f"\n🤖 【AI 整合回答】\n")
print(answer)
print()

# 步驟 4: 示範使用元數據過濾的 Chain
print("=" * 70)
print("步驟 4: 只查詢特定來源的 Chain")
print("=" * 70)

specific_question = "路由器如何設定？"
print(f"❓ 問題: {specific_question}\n")

# 創建只查詢路由器手冊的檢索器
def retrieve_from_router_manual(query):
    docs = db.similarity_search(
        query,
        k=3,
        filter={"source": "路由器設定手冊.txt"}
    )
    return docs

# 建立特定來源的 Chain
specific_chain = (
    {
        "context": RunnablePassthrough() | retrieve_from_router_manual | format_docs_with_source,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

specific_answer = specific_chain.invoke(specific_question)
print(f"🤖 【只查路由器手冊的回答】\n")
print(specific_answer)
print()

print("=" * 70)
print("📚 多來源 RAG Chain 的優勢")
print("=" * 70)
print("✅ 可以同時查詢多個產品手冊")
print("✅ 自動標註資訊來源，方便追蹤")
print("✅ 可以用 metadata 過濾特定來源")
print("✅ 整合多個文件的資訊給出完整答案")
print("\n💡 這展示了 Chain 3 中學到的組合技巧在 RAG 中的應用！")